# Лабараторая работа № 3

## Напоминалка про что датасет(про машины)

In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
sns.set_theme()
df = pd.read_csv('car details v4.csv')
df = df.drop(columns=['Unnamed: 0'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                2059 non-null   object 
 1   Model               2059 non-null   object 
 2   Price               2059 non-null   int64  
 3   Year                2059 non-null   int64  
 4   Kilometer           2059 non-null   int64  
 5   Fuel Type           2059 non-null   object 
 6   Transmission        2059 non-null   object 
 7   Location            2059 non-null   object 
 8   Color               2059 non-null   object 
 9   Owner               2059 non-null   object 
 10  Seller Type         2059 non-null   object 
 11  Drivetrain          1923 non-null   object 
 12  Length              1995 non-null   float64
 13  Width               1995 non-null   float64
 14  Height              1995 non-null   float64
 15  Seating Capacity    1995 non-null   float64
 16  Fuel T

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,RWD,4735.0,1830.0,1795.0,7.0,55.0


## Подготовка датасета

Какой атрибут выступит в роли целевого класса?
Создадим ещё один атрибут, чтобы получить бинарную классификацию - 'Price Type' со значениями "expensive" and "cheap".
Будем прогназировать по характеристикам (марка машины, год выпуска, тип двигателя) - дорогой автомобить или дешёвый.

Пусть Дорогой автомобить от дешёвого отличает граница 1 000 000 👀

In [13]:
def text_atribut_statistic(column, name, pd):
    print(f'Атрибут: {name}')
    statistic = column.value_counts(normalize=True) * 100
    if (statistic.size > 3):
        with_name = statistic.head(3)
        other = pd.Series({'другие': 100 - with_name.sum()})
        statistic = pd.concat([with_name, other])
    result = statistic.round(2).astype(str) + '%'
    print(result)

In [14]:
df['Price Type'] = np.where(df['Price'] >= 1000000, 'expensive', 'cheap')
text_atribut_statistic(df['Price Type'], 'Тип цены', pd)

Атрибут: Тип цены
Price Type
cheap        56.73%
expensive    43.27%
Name: proportion, dtype: object


### Подготовка

In [39]:
from sklearn.model_selection import train_test_split

#преобразуем данные к числовому виду ('Make', 'Fuel Type')
label_encoder_make = LabelEncoder()
df['Make'] = label_encoder_make.fit_transform(df['Make'])

label_encoder_price = LabelEncoder()
df['Fuel Type'] = label_encoder_price.fit_transform(df['Price Type'])

#преобразование целевого класса
label_encoder_price = LabelEncoder()
train_labels = label_encoder_price.fit_transform(df['Price Type'])

train_data = df[['Year', 'Make', 'Kilometer', 'Fuel Type']].values

#разбиваем на данные для обучения модели и для теста 80:20
data_train, data_test, labels_train, labels_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

### Дерево решений

In [40]:
from sklearn.tree import DecisionTreeClassifier

#создание модели
clf_tree = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state=17)

# обучение
clf_tree.fit(data_train, labels_train)

# Прогноз
forecast_tree = clf_tree.predict(data_test)

### kNN

In [41]:
from sklearn.neighbors import KNeighborsClassifier

#создание
knn = KNeighborsClassifier(n_neighbors=5)

#обучение
knn.fit(data_train, labels_train)

#прогнозирование
forecast_knn = knn.predict(data_test)

## Сравнить полученные результаты

In [71]:
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

#дерево решений
accuracy_tree = accuracy_score(labels_test, forecast_tree)
precision_tree = precision_score(labels_test, forecast_tree, average='weighted')
recall_tree = recall_score(labels_test, forecast_tree, average='weighted')
f_measure_tree = f1_score(labels_test, forecast_tree, average='weighted')

In [72]:
#kNN
accuracy_knn = accuracy_score(labels_test, forecast_knn)
precision_knn = precision_score(labels_test, forecast_knn, average='weighted')
recall_knn = recall_score(labels_test, forecast_knn, average='weighted')
f_measure_knn = f1_score(labels_test, forecast_knn, average='weighted')

In [75]:
print("Характеристики для Дерева решений")
print(f"Accuracy: {accuracy_tree}")
print(f"Precision: {precision_tree}")
print(f"Recall: {recall_tree}")
print(f"F measure: {f_measure_tree}")

print("Характеристики для Knn")
print(f"Accuracy: {accuracy_knn}")
print(f"Precision: {precision_knn}")
print(f"Recall: {recall_knn}")
print(f"F measure: {f_measure_knn}")

Характеристики для Дерева решений
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F measure: 1.0
Характеристики для Knn
Accuracy: 0.6383495145631068
Precision: 0.634455108114759
Recall: 0.6383495145631068
F measure: 0.624324396164955


## Вывод

Дерево решений показало себя идеально, оно однозначно прогнозировало тип цены машины в датасете. Доля правильных ответов, точность, полнота и баланс между Precision и Recall в kNN - гораздо ниже. Дело в том, что она чувствительна и хорошо работает на небольших датасетах. В то время, как специализация дерева решений - большие